In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import ast
import re

In [ ]:
book_path = '/content/drive/MyDrive/DS_KLTN/crawl_data/book/'

In [ ]:
df = pd.read_csv(book_path+'book_data.csv')

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
# drop brand
df.drop(columns=['brand'], inplace = True)

# add row for current_seller
df['current_seller'] = df['current_seller'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['seller_id'] = df['current_seller'].apply(lambda x:x['id'])
df['seller_name'] = df['current_seller'].apply(lambda x:x['name'])
df.drop(columns=['current_seller'], inplace = True)

# add row for category
df['categories'] = df['categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['category_id'] = df['categories'].apply(lambda x:x['id'])
df['category_name'] = df['categories'].apply(lambda x:x['name'])
df.drop(columns=['categories'], inplace = True)

# add row for breadcrumbs
breadcrumbs_df = []
for i in range(len(df)):
    if pd.notna(df.iloc[i]['breadcrumbs']):
        list_data = ast.literal_eval(df.iloc[i]['breadcrumbs'])[0]
        row_data = {'id': df.iloc[i]['id']}  # Thêm id vào row_data
        for j, item in enumerate(list_data):
            row_data[f'category_name_{j+1}'] = item['name']
            row_data[f'category_id_{j+1}'] = item['category_id']
        breadcrumbs_df.append(row_data)
    else:
        breadcrumbs_df.append({'id': df.iloc[i]['id']})
breadcrumbs_df = pd.DataFrame(breadcrumbs_df)
df = pd.merge(df.drop(columns=['breadcrumbs']), breadcrumbs_df, on='id', how='left')


# add row attributes
df['attributes'] = df['attributes'].replace('[','')
df['attributes'] = df['attributes'].replace(']','')
df['attributes'] = df['attributes'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
attributes_df = df['attributes'].apply(pd.Series)
for index, row in attributes_df.iterrows():
    for item in row:
        if isinstance(item, dict):  # Kiểm tra nếu item là dictionary
            df.loc[index, item['name']] = item['value']
df.drop(columns=['attributes'], inplace=True)

# add row authors
df['authors'] = df['authors'].replace('[','')
df['authors'] = df['authors'].replace(']','')
df['authors'] = df['authors'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df['author_id'] = df['authors'].apply(lambda x: [item['id'] for item in x] if isinstance(x, list) else None)
df['author_name'] = df['authors'].apply(lambda x: [item['name'] for item in x] if isinstance(x, list) else None)
df.drop(columns=['authors'], inplace=True)

In [ ]:
df.iloc[:, 28:] = df.iloc[:, 28:].replace(['nan', 'na'], None)
df['author_id'] = df['author_id'].str.replace(r'[\[\]]', '', regex=True)
df['author_name'] = df['author_name'].str.replace(r'[\[\]]', '', regex=True)
for i in df.columns:
  if df[i].isna().sum()/len(df) > 0.9:
    print(i, ': ', df[i].isna().sum())
    df.drop(i, axis=1, inplace=True)

In [ ]:
def process_text(text):
    return re.sub(r'<.*?>', '', text)

In [ ]:
df['description'] = df['description'].apply(process_text)

In [ ]:
df.rename(columns={
    'Công ty phát hành': 'Publisher',
    'Dịch Giả': 'Translator',
    'Loại bìa': 'Cover Type',
    'Số trang': 'Number of Pages',
    'Nhà xuất bản': 'Publishing House',
    'Ngày xuất bản': 'Publishing Date',
    'Phiên bản sách': 'Book Edition',
}, inplace=True)

df.drop(columns=['Kích thước'], inplace=True)

In [ ]:
col = [
'id', 'name', 'price', 'discount', 'images',
'highlight', 'description','Publisher','Translator',
'Cover Type', 'Number of Pages', 'Publishing House',
'Publishing Date', 'Book Edition','rating_average',
'review_count', 'quantity_sold', 'seller_id','seller_name',
'category_id','category_name', 'category_name_1', 'category_id_1',
'category_name_2','category_id_2', 'category_name_3', 'category_id_3',
'category_name_4','category_id_4', 'category_name_5', 'category_id_5']

In [ ]:
df= df[col]

In [ ]:
df = df[df['id'].notnull()]

In [ ]:
df.to_csv('/content/drive/MyDrive/DS_KLTN/data/book.csv',index=False,encoding='utf-8-sig')

In [ ]:
df['Cover Type'].value_counts()